# Imports

In [23]:
import pandas as pd
import pyspark
import string
import time

from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import col, isnull, sum, split, when, lower, regexp_replace, trim

# Data Preparation (Pyspark)

In [24]:
#Load the dataset

start = time.time()

# Create a SparkSession
spark = SparkSession.builder.appName("ReadCSV").getOrCreate()

# Read the CSV file into a DataFrame
df0 = spark.read.format("csv").option("header", "true").load("originalData.csv")

end = time.time()

print('Demorou ', (end - start), " seconds")

Demorou  0.1201789379119873  seconds


In [25]:
#Drop the column "views", "features" and "id"
df0 = df0.drop(*["id", "features", "views"])

In [26]:
#Drop the rows that have nulls values in the "lyrics" column
df0 = df0.na.drop(subset=["lyrics"])

In [27]:
#Drop the years before 1900 and after 2022
df0 = df0.filter((df0.year >= 1900) & (df0.year <= 2022))

In [28]:
#Drop the rows that have nulls values in the "title" column
df0 = df0.na.drop(subset=["title"])

In [29]:
#Drop the rows that have nulls values in the "tag" column
df0 = df0.na.drop(subset=["tag"])

In [30]:
# Create a new column with Boolean values indicating whether each row is null or not
df_with_nulls = df0.select([sum(col(c).isNull().cast("int")).alias(c) for c in df0.columns])

# Display the result
df_with_nulls.show()

+-----+---+------+----+------+
|title|tag|artist|year|lyrics|
+-----+---+------+----+------+
|    0|  0|     0|   0|     0|
+-----+---+------+----+------+



In [31]:
df0 = df0.withColumn("decade", when((df0.year >= 1900) & (df0.year < 1910), "00s")
                              .when((df0.year >= 1910) & (df0.year < 1920), "10s")
                              .when((df0.year >= 1920) & (df0.year < 1930), "20s")
                              .when((df0.year >= 1930) & (df0.year < 1940), "30s")
                              .when((df0.year >= 1940) & (df0.year < 1950), "40s")
                              .when((df0.year >= 1950) & (df0.year < 1960), "50s")
                              .when((df0.year >= 1960) & (df0.year < 1980), "60s")
                              .when((df0.year >= 1970) & (df0.year < 1980), "70s")
                              .when((df0.year >= 1980) & (df0.year < 1990), "80s")
                              .when((df0.year >= 1990) & (df0.year < 2000), "90s")
                              .when((df0.year >= 2000) & (df0.year < 2010), "000s")
                              .when((df0.year >= 2010) & (df0.year < 2020), "010s")
                              .when(df0.year >= 2020, "020s")
                             .otherwise("unknown"))

In [10]:
df0.show()

+--------------------+---+---------+----+--------------------+------+
|               title|tag|   artist|year|              lyrics|decade|
+--------------------+---+---------+----+--------------------+------+
|           Killa Cam|rap|  Cam'ron|2004|   ""Opera Steve""}"|  000s|
|          Can I Live|rap|    JAY-Z|1996|[Produced by Irv ...|   90s|
|   Forgive Me Father|rap| Fabolous|2003|Maybe cause I'm e...|  000s|
|        Down and Out|rap|  Cam'ron|2004|      ""Kanye West""|  000s|
|              Fly In|rap|Lil Wayne|2005|             [Intro]|  000s|
|      Lollipop Remix|rap|Lil Wayne|2008|  ""Static Major""}"|  000s|
|          Im Not You|rap|   Clipse|2002|""Roscoe P. Coldc...|  000s|
|         Family Ties|rap|  Cam'ron|2004|     ""Lady Wray""}"|  000s|
|   Rockin and Rollin|rap|  Cam'ron|1998|           [Verse 1]|   90s|
|       Lord You Know|rap|  Cam'ron|2004|   ""Juelz Santana""|  000s|
|    Money On My Mind|rap|Lil Wayne|2005|             [Intro]|  000s|
|     Think Yall Kno

In [33]:
df0.select("lyrics").show(truncate=False)

+--------------------------------+
|lyrics                          |
+--------------------------------+
|""Opera Steve""}"               |
|[Produced by Irv Gotti]         |
|Maybe cause I'm eatin           |
|""Kanye West""                  |
|[Intro]                         |
|""Static Major""}"              |
|""Roscoe P. Coldchain""}"       |
|""Lady Wray""}"                 |
|[Verse 1]                       |
|""Juelz Santana""               |
|[Intro]                         |
|[Verse 1]                       |
|[Produced by DJ Premier]        |
|[Produced by Just Blaze]        |
|[Verse 1]                       |
|[Intro: Birdman]                |
|[Produced by Timbaland]         |
|""Juelz Santana""}"             |
|[Intro: Jay-Z, Kid Capri & Both]|
|""Juelz Santana""}"             |
+--------------------------------+
only showing top 20 rows



In [11]:
""" result = df0.filter((df0["title"].contains("Down and Out")) & (df0["artist"] == "Cam'ron")).select("lyrics").collect()
for row in result:
    print(row["lyrics"]) """

""Kanye West""


In [34]:
#Convert "lyrics" column to lowercase
df0 = df0.withColumn("lyrics", lower(df0["lyrics"]))

In [35]:
#Replace the words "verse", "1", "2", "3", "chorus", "bridge", "outro", "instrumental", "intro", "guitar", "solo" for ""
df0 = df0.withColumn("lyrics", regexp_replace(df0["lyrics"], r"verse |[1|2|3]|chorus|bridge|outro|instrumental|intro|guitar|solo", ""))

In [36]:
#Replace the words "[" and "]" for ""
df0 = df0.withColumn("lyrics", regexp_replace(df0["lyrics"], r"\[", ""))
df0 = df0.withColumn("lyrics", regexp_replace(df0["lyrics"], r"\]", ""))

In [37]:
#Replace the words "(", ")" and "-" for ""
df0 = df0.withColumn("lyrics", regexp_replace("lyrics", r"[^\w\d'\s]+", ""))
df0 = df0.withColumn("lyrics", regexp_replace("lyrics", "efil ym fo flah", ""))

In [38]:
#Remove the "\n" and replace for " "
df0 = df0.withColumn("lyrics", regexp_replace("lyrics", "\n", " "))

In [39]:
df0 = df0.withColumn("lyrics", trim(df0["lyrics"]))

In [40]:
# Create a new column with Boolean values indicating whether each row is null or not
df_with_nulls1 = df0.select([sum(col(c).isNull().cast("int")).alias(c) for c in df0.columns])

# Display the result
df_with_nulls1.show()

+-----+---+------+----+------+------+
|title|tag|artist|year|lyrics|decade|
+-----+---+------+----+------+------+
|    0|  0|     0|   0|     0|     0|
+-----+---+------+----+------+------+



In [22]:
df0.show()

+--------------------+---+---------+----+--------------------+------+
|               title|tag|   artist|year|              lyrics|decade|
+--------------------+---+---------+----+--------------------+------+
|           Killa Cam|rap|  Cam'ron|2004|         opera steve|  000s|
|          Can I Live|rap|    JAY-Z|1996|produced by irv g...|   90s|
|   Forgive Me Father|rap| Fabolous|2003|maybe cause i'm e...|  000s|
|        Down and Out|rap|  Cam'ron|2004|          kanye west|  000s|
|              Fly In|rap|Lil Wayne|2005|                    |  000s|
|      Lollipop Remix|rap|Lil Wayne|2008|        static major|  000s|
|          Im Not You|rap|   Clipse|2002|  roscoe p coldchain|  000s|
|         Family Ties|rap|  Cam'ron|2004|           lady wray|  000s|
|   Rockin and Rollin|rap|  Cam'ron|1998|                    |   90s|
|       Lord You Know|rap|  Cam'ron|2004|       juelz santana|  000s|
|    Money On My Mind|rap|Lil Wayne|2005|                    |  000s|
|     Think Yall Kno

In [ ]:
#df0 = df0.withColumn("lyrics", regexp_replace(df0["lyrics"], r'\[.*?\]', ''))


In [20]:
#Remove the "\n" and replace the "efil ym fo flah" for ""
df0 = df0.withColumn("lyrics", regexp_replace("lyrics", r"[^\w\d'\s]+", ""))
df0 = df0.withColumn("lyrics", regexp_replace("lyrics", "efil ym fo flah", ""))


In [21]:
df0.show()

+--------------------+---+---------+----+--------------------+------+
|               title|tag|   artist|year|              lyrics|decade|
+--------------------+---+---------+----+--------------------+------+
|           Killa Cam|rap|  Cam'ron|2004|         opera steve|  000s|
|          Can I Live|rap|    JAY-Z|1996|produced by irv g...|   90s|
|   Forgive Me Father|rap| Fabolous|2003|maybe cause i'm e...|  000s|
|        Down and Out|rap|  Cam'ron|2004|          kanye west|  000s|
|              Fly In|rap|Lil Wayne|2005|                    |  000s|
|      Lollipop Remix|rap|Lil Wayne|2008|        static major|  000s|
|          Im Not You|rap|   Clipse|2002|  roscoe p coldchain|  000s|
|         Family Ties|rap|  Cam'ron|2004|           lady wray|  000s|
|   Rockin and Rollin|rap|  Cam'ron|1998|                    |   90s|
|       Lord You Know|rap|  Cam'ron|2004|       juelz santana|  000s|
|    Money On My Mind|rap|Lil Wayne|2005|                    |  000s|
|     Think Yall Kno

# Data Visualization

In [3]:
data.head()

,title,tag,artist,year,views,features,lyrics,id
0,Killa Cam,rap,Cam'ron,2004,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",1
1,Can I Live,rap,JAY-Z,1996,468624,{},"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...",3
2,Forgive Me Father,rap,Fabolous,2003,4743,{},Maybe cause I'm eatin\nAnd these bastards fien...,4
3,Down and Out,rap,Cam'ron,2004,144404,"{""Cam\\'ron"",""Kanye West"",""Syleena Johnson""}",[Produced by Kanye West and Brian Miller]\n\n[...,5
4,Fly In,rap,Lil Wayne,2005,78271,{},"[Intro]\nSo they ask me\n""Young boy\nWhat you ...",6


In [6]:
data.tail()

,title,tag,artist,year,views,features,lyrics,id
5913406,Raise Our Hands,pop,"Culture Code, Pag & Mylo",2016,3,"{Elex,""Culture Code / Pag & Mylo""}",[Verse 1]\nHere our purpose feels alive\nWe ar...,7882845
5913407,CEO,rap,Antropolita,2022,5,{},Jestem CEO w tym\nTo jara twoją bitch\nNikt na...,7882846
5913408,JokAir - Big Drip ft. ARÓN LineMa Traducción ...,rap,Genius Traducciones al Espaol,2022,1,"{""Genius Traducciones al Español""}",[Instrumental],7882847
5913409,New Number,country,Alana Springsteen,2022,1,{},"[Verse 1]\nYou need a new number, one that ain...",7882848
5913410,Gone,rap,VAPO,2022,2,"{""Loco (로꼬)""}",[Instrumental],7882849


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5913411 entries, 0 to 5913410
Data columns (total 8 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   title     object
 1   tag       object
 2   artist    object
 3   year      int64 
 4   views     int64 
 5   features  object
 6   lyrics    object
 7   id        int64 
dtypes: int64(3), object(5)
memory usage: 360.9+ MB


In [8]:
data.describe()

,year,views,id
count,5.913411e+06,5.913411e+06,5.913411e+06
mean,2.010663e+03,2.670159e+03,4.028247e+06
std,4.430131e+01,4.409858e+04,2.296420e+06
min,1.000000e+00,0.000000e+00,1.000000e+00
25%,2.010000e+03,1.600000e+01,1.821168e+06
50%,2.016000e+03,6.400000e+01,4.161274e+06
75%,2.019000e+03,3.490000e+02,6.007914e+06
max,2.100000e+03,2.335142e+07,7.882849e+06


In [10]:
data_tag = data['tag'].value_counts()
data_tag

pop        2519256
rap        1962010
rock        892220
rb          225342
misc        208714
country     105869
Name: tag, dtype: int64

In [11]:
data_year = data['year'].value_counts()
data_year

2020    683728
2021    630730
2019    576789
2015    521246
2018    481585
         ...  
1100         1
1437         1
936          1
1769         1
615          1
Name: year, Length: 747, dtype: int64